In [1]:
# Implementacion de la arquitectura VGG
# Very Deep Convolutional Networks for Large-Scale Image Recognition
# Existen diferentes versiones de VGG, en este caso nos centraremos en la version del paper D

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim

# Barra progeso durante entrenamiento
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.is_available())

True


In [4]:
# Aunque en el paper original no se normalizaban los subconjuntos de imágenes (batches)
# lo incluiremos ya que mejora el rendimiento del modelo durante el entrenamiento
# Sin añadir normalización, inicialmente la precisión del modelo era de 9% y permanecía constante (se paró a las 10 épocas)
# tras la normalización, la precisión inicial fue de 31%
class VGG(nn.Module):

    def __init__(self, num_clases):

        super(VGG, self).__init__()

        self.activacion = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size = (2, 2), stride = (2, 2))

        # Grupo 1
        self.stack_conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm1 = nn.BatchNorm2d(num_features = 64)
        self.stack_conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm2 = self.norm1

        # Grupo 2
        self.stack_conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm3 = nn.BatchNorm2d(num_features = 128)
        self.stack_conv4 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm4 = self.norm3

        # Grupo 3
        self.stack_conv5 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm5 = nn.BatchNorm2d(num_features = 256)
        self.stack_conv6 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm6 = self.norm5
        self.stack_conv7 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm7 = self.norm6

        # Grupo 4
        self.stack_conv8 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm8 = nn.BatchNorm2d(num_features = 512)
        self.stack_conv9 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm9 = self.norm8
        self.stack_conv10 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm10 = self.norm9

        # Grupo 5
        self.stack_conv11 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm11 = nn.BatchNorm2d(num_features = 512)
        self.stack_conv12 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm12= self.norm11
        self.stack_conv13 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.norm13 = self.norm12

        self.fc1 = nn.Linear(in_features = 7 * 7 * 512, out_features = 4096)
        self.fc2 = nn.Linear(in_features = 4096, out_features = 4096)
        self.fc3 = nn.Linear(in_features = 4096, out_features = num_clases)

    def forward(self, x):

        # Grupo 1
        x = self.activacion(self.norm1(self.stack_conv1(x)))
        x = self.activacion(self.norm2(self.stack_conv2(x)))
        
        # Grupo 2
        x = self.pool(x)
        x = self.activacion(self.norm3(self.stack_conv3(x)))
        x = self.activacion(self.norm4(self.stack_conv4(x)))
        
        # Grupo 3
        x = self.pool(x)
        x = self.activacion(self.norm5(self.stack_conv5(x)))
        x = self.activacion(self.norm6(self.stack_conv6(x)))
        x = self.activacion(self.norm7(self.stack_conv7(x)))

        # Grupo 4
        x = self.pool(x)
        x = self.activacion(self.norm8(self.stack_conv8(x)))
        x = self.activacion(self.norm9(self.stack_conv9(x)))
        x = self.activacion(self.norm10(self.stack_conv10(x)))

        # Grupo 5
        x = self.pool(x)
        x = self.activacion(self.norm11(self.stack_conv11(x)))
        x = self.activacion(self.norm12(self.stack_conv12(x)))
        x = self.activacion(self.norm13(self.stack_conv13(x)))

        # Pooling
        x = self.pool(x)

        # Aplanamos la matriz a un vector
        x = x.reshape(x.shape[0], -1)

        # FC 1
        x = self.activacion(self.fc1(x))
        x = F.dropout(input = x, p = 0.5)
        
        # FC 2
        x = self.activacion(self.fc2(x))
        x = F.dropout(input = x, p = 0.5)

        # FC 3
        x = self.fc3(x)

        return x

In [5]:
# Hiperparámetros
num_clases = 10 # Para CIFAR-10 tenemos 10 clases
batch_size = 16
learning_rate = 1e-2
num_epocas = 40

In [6]:
# Vamos a descargar el dataset de CIFAR10

transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
                         std  = [ 0.229, 0.224, 0.225 ]),
    ])

train_dataset = datasets.CIFAR10(root = 'dataset/', train = True, transform = transform, download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = datasets.CIFAR10(root = 'dataset/', train = False, transform = transform, download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
print(train_dataset)
print(test_dataset)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [8]:
modelo = VGG(num_clases = num_clases).to(device)

In [9]:
# Loss function, para clasificaciones de una sola etiqueta (y) como en este caso podemos utilizar Cross Entropy Loss
func_perdida = nn.CrossEntropyLoss()

# No es recomendable utilizar Adam para entrenar VGG debido a la cantidad de parámetros que hay
# optimizador = optim.Adam(params = modelo.parameters(), lr = learning_rate)
optimizador = optim.SGD(params = modelo.parameters(), lr = learning_rate)

In [10]:
cont = 0

for epoca in range(num_epocas):

    correcto = 0
    
    for batch_idx, (x, y) in enumerate(train_loader):

        x = x.to(device = device)
        y = y.to(device = device)

        if (cont == 0):
            
            print(x.shape)
            cont += 1

        # forward
        scores = modelo(x)
        loss = func_perdida(scores, y)

        # Backpropagation
        # Zero your gradients for every batch!
        # Ponemos el gradiente a 0 para cada batch para que no almacene las derivadas de las anteriores capas
        optimizador.zero_grad() 
        loss.backward()

        # Descenso del gradiente o pasos del algoritmo ADAM
        optimizador.step()
        
        #correcto += (scores == y).float().sum()
        _,pred_label = torch.max(scores, dim = 1)
        correcto += (pred_label == y).float().sum()
    
    # Podemos ajustar el learning rate conforme vamos entrenando el modelo, inicialmente el modelo
    # requerirá pasos más grandes para aprender con mayor velocidad, conforme se encuentre a un mínimo
    # los pasos deben reducirse con la intención de que no se aleje de dicho mínimo.
    # En este caso cada 8 épocas estaríamos reduciendo el learning rate 10
    if epoca % 8 == 0:

      learning_rate /= 10 
      optimizador.param_groups[0]['lr'] = learning_rate

    print(f"Epoca [{epoca+1}/{num_epocas}]\tPrecision {((100 * correcto) / len(train_dataset))}")

torch.Size([16, 3, 224, 224])


RuntimeError: CUDA error: the launch timed out and was terminated

In [ ]:
# Comprobar precisión del modelo en entrenamiento y validacion
def comprobar_precision(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    # Para evaluar el modelo no necesitamos calcular el gradiente
    with torch.no_grad(): 
    
        for x, y in loader:
    
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Tenemos {num_correct} / {num_samples} con una precision de {float(num_correct) / float(num_samples) * 100:.2f}")

    model.train()